<a href="https://colab.research.google.com/github/MuhammadIrzam447/MultiModel/blob/master/Valid_54.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!gdown https://drive.google.com/uc?id=1ytJdl1n4e2Y-F8a8mJab0S_4TgyrkE-4

Downloading...
From: https://drive.google.com/uc?id=1ytJdl1n4e2Y-F8a8mJab0S_4TgyrkE-4
To: /content/joint_representations.zip
100% 1.39G/1.39G [00:31<00:00, 43.5MB/s]


In [ ]:
!unzip /content/joint_representations.zip

# New Section

In [ ]:
!pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# # the model name
# model_name = "google/vit-base-patch16-224"
# # load the image processor
# image_processor = ViTImageProcessor.from_pretrained(model_name)
# # loading the pre-trained model
# model = ViTForImageClassification.from_pretrained(model_name).to(device)

In [ ]:
# load the best model, change the checkpoint number to the best checkpoint
# if the last checkpoint is the best, then ignore this cell
best_checkpoint = 7000
# best_checkpoint = 150
model = ViTForImageClassification.from_pretrained(f"./vit-base-food/checkpoint-{best_checkpoint}").to(device)
# model = ViTForImageClassification.from_pretrained(f"./vit-base-skin-cancer/checkpoint-{best_checkpoint}").to(device)

In [ ]:
# import urllib.parse as parse
# import os

# # a function to determine whether a string is a URL or not
# def is_url(string):
#     try:
#         result = parse.urlparse(string)
#         return all([result.scheme, result.netloc, result.path])
#     except:
#         return False

# # a function to load an image
# def load_image(image_path):
#     if is_url(image_path):
#         return Image.open(requests.get(image_path, stream=True).raw)
#     elif os.path.exists(image_path):
#         return Image.open(image_path)

In [ ]:
# def get_prediction(model, url_or_path):
#   # load the image
#   img = load_image(url_or_path)
#   # preprocessing the image
#   pixel_values = image_processor(img, return_tensors="pt")["pixel_values"].to(device)
#   # perform inference
#   output = model(pixel_values)
#   # get the label id and return the class name
#   return model.config.id2label[int(output.logits.softmax(dim=1).argmax())]

In [ ]:
# get_prediction(model, "http://images.cocodataset.org/test-stuff2017/000000000128.jpg")

# Loading our Dataset

In [ ]:
# from datasets import load_dataset

# # download & load the dataset
# ds = load_dataset("food101")

## Loading a Custom Dataset using `ImageFolder`
Run the three below cells to load a custom dataset (that's not in the Hub) using `ImageFolder`

In [ ]:
# import requests
# from tqdm import tqdm

# def get_file(url):
#   response = requests.get(url, stream=True)
#   total_size = int(response.headers.get('content-length', 0))
#   filename = None
#   content_disposition = response.headers.get('content-disposition')
#   if content_disposition:
#       parts = content_disposition.split(';')
#       for part in parts:
#           if 'filename' in part:
#               filename = part.split('=')[1].strip('"')
#   if not filename:
#       filename = os.path.basename(url)
#   block_size = 1024 # 1 Kibibyte
#   tqdm_bar = tqdm(total=total_size, unit='iB', unit_scale=True)
#   with open(filename, 'wb') as file:
#       for data in response.iter_content(block_size):
#           tqdm_bar.update(len(data))
#           file.write(data)
#   tqdm_bar.close()
#   print(f"Downloaded {filename} ({total_size} bytes)")
#   return filename

In [ ]:
# import zipfile
# import os

# def download_and_extract_dataset():
#   # dataset from https://github.com/udacity/dermatologist-ai
#   # 5.3GB
#   train_url = "https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/train.zip"
#   # 824.5MB
#   valid_url = "https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/valid.zip"
#   # 5.1GB
#   test_url  = "https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/test.zip"
#   for i, download_link in enumerate([valid_url, train_url, test_url]):
#     data_dir = get_file(download_link)
#     print("Extracting", download_link)
#     with zipfile.ZipFile(data_dir, "r") as z:
#       z.extractall("data")
#     # remove the temp file
#     os.remove(data_dir)

# # comment the below line if you already downloaded the dataset
# download_and_extract_dataset()

In [ ]:
from datasets import load_dataset

# load the custom dataset
ds = load_dataset("imagefolder", data_dir="/content/hateful_ViT1")

# 100% Missing Modality Dataset

In [ ]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/hateful_ViT1")

In [ ]:
def filename_ends_with_3(example):
    return example["file_name"].endswith("_3.png")

In [ ]:
ds = ds.filter(filename_ends_with_3)
print(ds["train"][0:5])

# Exploring the Data

In [ ]:
ds

In [ ]:
labels = ds["train"].features["label"]
labels

In [ ]:
labels.int2str(ds["train"][532]["label"])

In [ ]:
# import random
# import matplotlib.pyplot as plt

# def show_image_grid(dataset, split, grid_size=(4,4)):
#     # Select random images from the given split
#     indices = random.sample(range(len(dataset[split])), grid_size[0]*grid_size[1])
#     images = [dataset[split][i]["image"] for i in indices]
#     labels = [dataset[split][i]["label"] for i in indices]

#     # Display the images in a grid
#     fig, axes = plt.subplots(nrows=grid_size[0], ncols=grid_size[1], figsize=(8,8))
#     for i, ax in enumerate(axes.flat):
#         ax.imshow(images[i])
#         ax.axis('off')
#         ax.set_title(ds["train"].features["label"].int2str(labels[i]))

#     plt.show()

In [ ]:
# show_image_grid(ds, "train")

# Preprocessing the Data

In [ ]:
def transform(examples):
  # convert all images to RGB format, then preprocessing it
  # using our image processor
  inputs = image_processor([img.convert("RGB") for img in examples["image"]], return_tensors="pt")
  # we also shouldn't forget about the labels
  inputs["labels"] = examples["label"]
  return inputs

In [ ]:
# use the with_transform() method to apply the transform to the dataset on the fly during training
dataset = ds.with_transform(transform)

In [ ]:
for item in dataset["train"]:
  print(item["pixel_values"].shape)
  print(item["labels"])
  break

In [ ]:
# extract the labels for our dataset
labels = ds["train"].features["label"].names
labels

In [ ]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

# Defining the Metrics

In [ ]:
from evaluate import load
import numpy as np
from sklearn.metrics import roc_auc_score

# load the accuracy and f1 metrics from the evaluate module
accuracy = load("accuracy")
f1 = load("f1")

def compute_metrics(eval_pred):
  # compute the accuracy and f1 scores & return them
  accuracy_score = accuracy.compute(predictions=np.argmax(eval_pred.predictions, axis=1), references=eval_pred.label_ids)
  f1_score = f1.compute(predictions=np.argmax(eval_pred.predictions, axis=1), references=eval_pred.label_ids, average="macro")

  auroc_score = roc_auc_score(eval_pred.label_ids, np.argmax(eval_pred.predictions, axis=1))
  print(f"AUROC Score: {auroc_score:.4f}")

  return {**accuracy_score, **f1_score}

# Training the Model

In [ ]:
# # load the ViT model
# model = ViTForImageClassification.from_pretrained(
#     model_name,
#     num_labels=len(labels),
#     id2label={str(i): c for i, c in enumerate(labels)},
#     label2id={c: str(i) for i, c in enumerate(labels)},
#     ignore_mismatched_sizes=True,
# )

In [ ]:
# !pip install accelerate -U

In [ ]:
# !pip install transformers[torch]

In [ ]:
# from transformers import TrainingArguments

# training_args = TrainingArguments(
#   output_dir="/content/output", # output directory
#   per_device_train_batch_size=32, # batch size per device during training
#   evaluation_strategy="steps",    # evaluation strategy to adopt during training
#   num_train_epochs=10,             # total number of training epochs
#   # fp16=True,                    # use mixed precision
#   save_steps=1000,                # number of update steps before saving checkpoint
#   eval_steps=1000,                # number of update steps before evaluating
#   logging_steps=1000,             # number of update steps before logging
#   # save_steps=50,
#   # eval_steps=50,
#   # logging_steps=50,
#   save_total_limit=2,             # limit the total amount of checkpoints on disk
#   remove_unused_columns=False,    # remove unused columns from the dataset
#   push_to_hub=False,              # do not push the model to the hub
#   report_to='tensorboard',        # report metrics to tensorboard
#   load_best_model_at_end=True,    # load the best model at the end of training
# )


In [ ]:
dataset

In [ ]:
# from transformers import Trainer

# trainer = Trainer(
#     model=model,                        # the instantiated 🤗 Transformers model to be trained
#     args=training_args,                 # training arguments, defined above
#     data_collator=collate_fn,           # the data collator that will be used for batching
#     compute_metrics=compute_metrics,    # the metrics function that will be used for evaluation
#     train_dataset=dataset["train"],     # training dataset
#     eval_dataset=dataset["test"],       # evaluation dataset
#     tokenizer=image_processor,          # the processor that will be used for preprocessing the images
# )

In [ ]:
# start training
trainer.train()

In [ ]:
# trainer.evaluate(dataset["test"])
trainer.evaluate()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil

# Define the source folder path (in Colab)
source_folder_path = '/content/output/checkpoint-7000'

# Define the destination folder path (in Google Drive)
destination_folder_path = "/content/drive/MyDrive/Colab Notebooks/Hateful-Memes/Vit/checkpoint-7000"

# Remove the existing destination folder (if it exists)
if os.path.exists(destination_folder_path):
    shutil.rmtree(destination_folder_path)

# Copy the folder
shutil.copytree(source_folder_path, destination_folder_path)

In [ ]:
# start tensorboard
# %load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir ./vit-base-food/runs

## Alternatively: Training using PyTorch Loop
Run the two below cells to fine-tune using a regular PyTorch loop if you want.

In [ ]:
# Training loop
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW
from torch.utils.data import DataLoader

batch_size = 32

train_dataset_loader = DataLoader(dataset["train"], collate_fn=collate_fn, batch_size=batch_size, shuffle=True)
valid_dataset_loader = DataLoader(dataset["validation"], collate_fn=collate_fn, batch_size=batch_size, shuffle=True)

# define the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

log_dir = "./image-classification/tensorboard"
summary_writer = SummaryWriter(log_dir=log_dir)

num_epochs = 3
model = model.to(device)
# print some statistics before training
# number of training steps
n_train_steps = num_epochs * len(train_dataset_loader)
# number of validation steps
n_valid_steps = len(valid_dataset_loader)
# current training step
current_step = 0
# logging, eval & save steps
save_steps = 1000

def compute_metrics(eval_pred):
  accuracy_score = accuracy.compute(predictions=eval_pred.predictions, references=eval_pred.label_ids)
  f1_score = f1.compute(predictions=eval_pred.predictions, references=eval_pred.label_ids, average="macro")
  return {**accuracy_score, **f1_score}

In [ ]:
for epoch in range(num_epochs):
    # set the model to training mode
    model.train()
    # initialize the training loss
    train_loss = 0
    # initialize the progress bar
    progress_bar = tqdm(range(current_step, n_train_steps), "Training", dynamic_ncols=True, ncols=80)
    for batch in train_dataset_loader:
      if (current_step+1) % save_steps == 0:
        ### evaluation code ###
        # evaluate on the validation set
        # if the current step is a multiple of the save steps
        print()
        print(f"Validation at step {current_step}...")
        print()
        # set the model to evaluation mode
        model.eval()
        # initialize our lists that store the predictions and the labels
        predictions, labels = [], []
        # initialize the validation loss
        valid_loss = 0
        for batch in valid_dataset_loader:
            # get the batch
            pixel_values = batch["pixel_values"].to(device)
            label_ids = batch["labels"].to(device)
            # forward pass
            outputs = model(pixel_values=pixel_values, labels=label_ids)
            # get the loss
            loss = outputs.loss
            valid_loss += loss.item()
            # free the GPU memory
            logits = outputs.logits.detach().cpu()
            # add the predictions to the list
            predictions.extend(logits.argmax(dim=-1).tolist())
            # add the labels to the list
            labels.extend(label_ids.tolist())
        # make the EvalPrediction object that the compute_metrics function expects
        eval_prediction = EvalPrediction(predictions=predictions, label_ids=labels)
        # compute the metrics
        metrics = compute_metrics(eval_prediction)
        # print the stats
        print()
        print(f"Epoch: {epoch}, Step: {current_step}, Train Loss: {train_loss / save_steps:.4f}, " +
              f"Valid Loss: {valid_loss / n_valid_steps:.4f}, Accuracy: {metrics['accuracy']}, " +
              f"F1 Score: {metrics['f1']}")
        print()
        # log the metrics
        summary_writer.add_scalar("valid_loss", valid_loss / n_valid_steps, global_step=current_step)
        summary_writer.add_scalar("accuracy", metrics["accuracy"], global_step=current_step)
        summary_writer.add_scalar("f1", metrics["f1"], global_step=current_step)
        # save the model
        model.save_pretrained(f"./vit-base-food/checkpoint-{current_step}")
        image_processor.save_pretrained(f"./vit-base-food/checkpoint-{current_step}")
        # get the model back to train mode
        model.train()
        # reset the train and valid loss
        train_loss, valid_loss = 0, 0
      ### training code below ###
      # get the batch & convert to tensor
      pixel_values = batch["pixel_values"].to(device)
      labels = batch["labels"].to(device)
      # forward pass
      outputs = model(pixel_values=pixel_values, labels=labels)
      # get the loss
      loss = outputs.loss
      # backward pass
      loss.backward()
      # update the weights
      optimizer.step()
      # zero the gradients
      optimizer.zero_grad()
      # log the loss
      loss_v = loss.item()
      train_loss += loss_v
      # increment the step
      current_step += 1
      progress_bar.update(1)
      # log the training loss
      summary_writer.add_scalar("train_loss", loss_v, global_step=current_step)


# Performing Inference

In [ ]:
# load the best model, change the checkpoint number to the best checkpoint
# if the last checkpoint is the best, then ignore this cell
best_checkpoint = 7000
# best_checkpoint = 150
model = ViTForImageClassification.from_pretrained(f"./vit-base-food/checkpoint-{best_checkpoint}").to(device)
# model = ViTForImageClassification.from_pretrained(f"./vit-base-skin-cancer/checkpoint-{best_checkpoint}").to(device)

In [ ]:
get_prediction(model, "https://images.pexels.com/photos/858496/pexels-photo-858496.jpeg?auto=compress&cs=tinysrgb&w=600&lazy=load")

In [ ]:
def get_prediction_probs(model, url_or_path, num_classes=3):
    # load the image
    img = load_image(url_or_path)
    # preprocessing the image
    pixel_values = image_processor(img, return_tensors="pt")["pixel_values"].to(device)
    # perform inference
    output = model(pixel_values)
    # get the top k classes and probabilities
    probs, indices = torch.topk(output.logits.softmax(dim=1), k=num_classes)
    # get the class labels
    id2label = model.config.id2label
    classes = [id2label[idx.item()] for idx in indices[0]]
    # convert the probabilities to a list
    probs = probs.squeeze().tolist()
    # create a dictionary with the class names and probabilities
    results = dict(zip(classes, probs))
    return results

In [ ]:
# example 1
get_prediction_probs(model, "https://images.pexels.com/photos/406152/pexels-photo-406152.jpeg?auto=compress&cs=tinysrgb&w=600")

In [ ]:
# example 2
get_prediction_probs(model, "https://images.pexels.com/photos/920220/pexels-photo-920220.jpeg?auto=compress&cs=tinysrgb&w=600")

In [ ]:
# example 3
get_prediction_probs(model, "https://images.pexels.com/photos/3338681/pexels-photo-3338681.jpeg?auto=compress&cs=tinysrgb&w=600")

In [ ]:
# example 4
get_prediction_probs(model, "https://images.pexels.com/photos/806457/pexels-photo-806457.jpeg?auto=compress&cs=tinysrgb&w=600", num_classes=10)

In [ ]:
get_prediction_probs(model, "https://images.pexels.com/photos/1624487/pexels-photo-1624487.jpeg?auto=compress&cs=tinysrgb&w=600")

# Validation


In [1]:
!pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
model = ViTForImageClassification.from_pretrained(f"/content/drive/MyDrive/Colab Notebooks/Hateful-Memes/Vit/checkpoint-7000").to(device)
image_processor = ViTImageProcessor.from_pretrained(f"/content/drive/MyDrive/Colab Notebooks/Hateful-Memes/Vit/checkpoint-7000")

loading configuration file /content/drive/MyDrive/Colab Notebooks/Hateful-Memes/Vit/checkpoint-7000/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "0",
    "1": "1"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "0": "0",
    "1": "1"
  },
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "problem_type": "single_label_classification",
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.33.1"
}

loading weights file /content/drive/MyDrive/Colab Notebooks/Hateful-Memes/Vit/checkpoint-7000/pytorch_model.bin
All model checkpoint weights were used when ini

In [7]:
from datasets import load_dataset

ds = load_dataset("imagefolder", data_dir="/content/mulitmodal/test", split="train")

Resolving data files:   0%|          | 0/4000 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
ds

Dataset({
    features: ['image', 'label'],
    num_rows: 4000
})

In [9]:
# Get the list of column names (fields)
column_names = ds.column_names

# Print the list of column names
print(column_names)

['image', 'label']


In [10]:
import os
def filename_ends_with_4(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_4.png")

In [11]:
ds = ds.filter(filename_ends_with_4)

Filter:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [12]:
ds

Dataset({
    features: ['image', 'label'],
    num_rows: 2000
})

In [ ]:
for i in range(150,500):
    sample = ds[i]
    img = sample["image"]
    filename = os.path.basename(img.filename)
    print(f"File Name: {filename}")

In [16]:
labels = ds.features["label"]
labels

ClassLabel(names=['0', '1'], id=None)

In [17]:
labels.int2str(ds[532]["label"])

'0'

In [18]:
def transform(examples):
  # convert all images to RGB format, then preprocessing it
  # using our image processor
  inputs = image_processor([img.convert("RGB") for img in examples["image"]], return_tensors="pt")
  # we also shouldn't forget about the labels
  inputs["labels"] = examples["label"]
  return inputs

In [19]:
# use the with_transform() method to apply the transform to the dataset on the fly during training
dataset = ds.with_transform(transform)

In [20]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 2000
})

In [21]:
for item in dataset:
  print(item["pixel_values"].shape)
  print(item["labels"])
  break

torch.Size([3, 224, 224])
0


In [22]:
# extract the labels for our dataset
labels = ds.features["label"].names
labels

['0', '1']

In [23]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [ ]:
from evaluate import load
import numpy as np
from sklearn.metrics import roc_auc_score

# load the accuracy and f1 metrics from the evaluate module
accuracy = load("accuracy")
f1 = load("f1")

def compute_metrics(eval_pred):
  # compute the accuracy and f1 scores & return them
  accuracy_score = accuracy.compute(predictions=np.argmax(eval_pred.predictions, axis=1), references=eval_pred.label_ids)
  f1_score = f1.compute(predictions=np.argmax(eval_pred.predictions, axis=1), references=eval_pred.label_ids, average="macro")

  auroc_score = roc_auc_score(eval_pred.label_ids, np.argmax(eval_pred.predictions, axis=1))
  print(f"AUROC Score: {auroc_score:.4f}")

  return {**accuracy_score, **f1_score}

In [24]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 2000
})

In [25]:
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW
from torch.utils.data import DataLoader

batch_size = 32

# train_dataset_loader = DataLoader(dataset["train"], collate_fn=collate_fn, batch_size=batch_size, shuffle=True)
valid_dataset_loader = DataLoader(dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=True)

# define the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

In [ ]:
# log_dir = "./image-classification/tensorboard"
# summary_writer = SummaryWriter(log_dir=log_dir)

# num_epochs = 3


# number of training steps
# n_train_steps = num_epochs * len(train_dataset_loader)

# # number of validation steps
# n_valid_steps = len(valid_dataset_loader)
# # current training step

# current_step = 0
# # logging, eval & save steps
# save_steps = 1000

# def compute_metrics(eval_pred):
#   accuracy_score = accuracy.compute(predictions=eval_pred.predictions, references=eval_pred.label_ids)
#   f1_score = f1.compute(predictions=eval_pred.predictions, references=eval_pred.label_ids, average="macro")
#   auroc_score = roc_auc_score(eval_pred.label_ids, np.argmax(eval_pred.predictions, axis=1))
#   print("Accuracy: ", accuracy_score)
#   print("F1:",f1_score)
#   print("AUROC: ", auroc_score)
#   return {**accuracy_score, **f1_score}

In [26]:
model = model.to(device)
model.eval()
predictions, labels = [], []
valid_loss = 0
for batch in valid_dataset_loader:
    pixel_values = batch["pixel_values"].to(device)
    label_ids = batch["labels"].to(device)

    outputs = model(pixel_values=pixel_values, labels=label_ids)

    loss = outputs.loss
    valid_loss += loss.item()

    logits = outputs.logits.detach().cpu()

    predictions.extend(logits.argmax(dim=-1).tolist())
    labels.extend(label_ids.tolist())

In [28]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions, average='weighted')
recall = recall_score(labels, predictions, average='weighted')
f1 = f1_score(labels, predictions, average='weighted')
AUROC_score = roc_auc_score(labels, predictions)

In [29]:
print("Accuracy: ", accuracy)
print("precision: ", precision)
print("f1_score: ", f1)
print("AUROC_score: ", AUROC_score)

Accuracy:  0.7175
precision:  0.7139426389922545
f1_score:  0.7151643090757657
AUROC_score:  0.6913333333333334


In [30]:
cm = confusion_matrix(labels, predictions)
print(cm)

[[995 255]
 [310 440]]
